In [1]:
from dotenv import load_dotenv
import os 
import pymongo
import pandas as pd
from tqdm.auto import tqdm

load_dotenv()

True

In [2]:
client = pymongo.MongoClient(os.getenv('MONGO_CONNECTION_STRING'))
db = client["monin"]
collection = db["drinks"]

In [3]:
collection.find_one()

{'_id': ObjectId('67cdeca2d61baaa7449cc9f0'),
 'name': 'Hawaiian Pearl Bubble Tea',
 'recipie': ['30 mL Monin Hawaiian Island Syrup',
  '150 mL tea',
  '60 mL club soda',
  '45 mL tapioca bobas',
  'Fill serving glass full of ice.',
  'Pour ingredients into serving glass in order listed.',
  'Stir gently to mix.',
  'Add garnish and serve.'],
 'category': 'bubble_teas',
 'image_url': 'https://monin.blob.core.windows.net/recipe/images/mrd/drink/5125b732-6dea-4287-a76e-fdb0588dc394-2.png',
 'drink_url': 'https://monin.us/products/hawaiian-pearl-bubble-tea',
 'taste': ['Citrusy', 'Sour'],
 'cluster': 7,
 'position': {'text': [-34.186474, -48.2288],
  'image': [-16.28633, -11.720665],
  'combined': [-43.066746, -16.4077]}}

In [11]:
base_path = '../data/'
clustering_data = ['text_embeddings_data', 'image_embeddings_data', 'combined_embeddings_data']

for data in clustering_data:
  field_name = data.split('_')[0]
  df = pd.read_csv(os.path.join(base_path, f'{data}.csv'))[['drink_url', 'tsne_x', 'tsne_y', 'cluster']]
  
  updates = []
  for row in tqdm(df.itertuples()):
    url, x, y, cluster = row[1], row[2], row[3], row[4]
    updates.append(
      pymongo.UpdateMany(
        {'drink_url': url},
        {
          # "$unset": {'cluster': ""}, 
         "$set": {f'position.{field_name}': [x, y], f'cluster.{field_name}': cluster}
         }
      )
    )
    # break 
  
  if updates:
    collection.bulk_write(updates)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]